In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.set_option('display.max_rows',None)

In [2]:
!ls

 ashrae-energy-prediction.zip   merged_test.csv         train.csv
'Basic model.ipynb'	        merged_train.csv        weather_test.csv
 building_metadata.csv	        sample_submission.csv   weather_train.csv
 EDA.ipynb		        test.csv


### Reading data

In [3]:
train_df = pd.read_csv('train.csv')
print(train_df.shape)
train_df.head()

(20216100, 4)


,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [4]:
train_df.isna().sum()

building_id      0
meter            0
timestamp        0
meter_reading    0
dtype: int64

In [5]:
test_df = pd.read_csv('test.csv')
print(test_df.shape)
test_df.head()

(41697600, 4)


,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


In [6]:
test_df.isna().sum()

row_id         0
building_id    0
meter          0
timestamp      0
dtype: int64

**weather train data**

In [7]:
weather_train_df = pd.read_csv('weather_train.csv')
print(weather_train_df.shape)
weather_train_df.head()

(139773, 9)


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [8]:
weather_train_df.isna().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64

In [9]:
# weather_train_df['cloud_coverage'] = weather_train_df['cloud_coverage'].astype('category')
# weather_train_df['cloud_coverage'] = weather_train_df['cloud_coverage'].cat.codes

# weather_train_df['cloud_coverage'] = weather_train_df['cloud_coverage'].astype('category')
# weather_train_df['cloud_coverage'] = weather_train_df['cloud_coverage'].cat.codes

In [10]:
weather_train_df.interpolate(inplace=True)

In [11]:
weather_train_df.isna().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     1
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [12]:
weather_train_df.fillna(-1.0,inplace = True)

In [13]:
weather_train_df.isna().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [14]:
weather_test_df = pd.read_csv('weather_test.csv')
print(weather_test_df.shape)
weather_test_df.head()

(277243, 9)


,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6


In [15]:
weather_test_df.timestamp.nunique()

17520

In [16]:
test_df.timestamp.nunique()

17520

In [17]:
weather_test_df.isna().sum()

site_id                    0
timestamp                  0
air_temperature          104
cloud_coverage        140448
dew_temperature          327
precip_depth_1_hr      95588
sea_level_pressure     21265
wind_direction         12370
wind_speed               460
dtype: int64

In [18]:
weather_test_df.interpolate(inplace=True)

In [19]:
weather_test_df.isna().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     1
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [20]:
weather_test_df.fillna(-1.0,inplace = True)
weather_test_df.isna().sum()

site_id               0
timestamp             0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

**Building_meta data**

In [21]:
building_meatdata_Df = pd.read_csv('building_metadata.csv')
print(building_meatdata_Df.shape)
building_meatdata_Df.head()

(1449, 6)


,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [22]:
building_meatdata_Df.building_id.nunique()

1449

In [23]:
train_df.building_id.nunique()

1449

In [24]:
test_df.building_id.nunique()

1449

In [25]:
building_meatdata_Df.isna().sum()

site_id           0
building_id       0
primary_use       0
square_feet       0
year_built      774
floor_count    1094
dtype: int64

**Change year built and floor count as cat values**

In [26]:
building_meatdata_Df['year_built'] = building_meatdata_Df['year_built'].astype('category')
building_meatdata_Df['year_built'] = building_meatdata_Df['year_built'].cat.codes

building_meatdata_Df['floor_count'] = building_meatdata_Df['floor_count'].astype('category')
building_meatdata_Df['floor_count'] = building_meatdata_Df['floor_count'].cat.codes

In [27]:
building_meatdata_Df.isna().sum()

site_id        0
building_id    0
primary_use    0
square_feet    0
year_built     0
floor_count    0
dtype: int64

In [28]:
building_meatdata_Df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,106,-1
1,0,1,Education,2720,102,-1
2,0,2,Education,5376,89,-1
3,0,3,Education,23685,100,-1
4,0,4,Education,116607,73,-1


   **merge building metadata over train_df**

In [29]:
train_df = pd.merge(train_df,building_meatdata_Df,on='building_id',how='left')
print(train_df.shape)
train_df.head()

(20216100, 9)


,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,106,-1
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,102,-1
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,89,-1
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,100,-1
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,73,-1


In [30]:
train_df.isna().sum()

building_id      0
meter            0
timestamp        0
meter_reading    0
site_id          0
primary_use      0
square_feet      0
year_built       0
floor_count      0
dtype: int64

In [31]:
train_df.timestamp.nunique()

8784

In [32]:
weather_train_df.timestamp.nunique()

8784

In [33]:
test_df = pd.merge(test_df,building_meatdata_Df,on='building_id',how='left')
print(test_df.shape)
test_df.head()

(41697600, 9)


,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,106,-1
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,102,-1
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,89,-1
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,100,-1
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,73,-1


In [34]:
test_df.isna().sum()

row_id         0
building_id    0
meter          0
timestamp      0
site_id        0
primary_use    0
square_feet    0
year_built     0
floor_count    0
dtype: int64

In [35]:
weather_train_df.head(2)

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,4.0,21.1,-1.0,1020.2,70.0,1.5


In [36]:
train_df.head(2)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,106,-1
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,102,-1


In [37]:
print(train_df.site_id.nunique(),weather_train_df.site_id.nunique())
print(train_df.timestamp.nunique(),weather_train_df.timestamp.nunique())

16 16
8784 8784


**merge weather data over train_df**

In [38]:
train_df = pd.merge(train_df,weather_train_df,on=['site_id','timestamp'],how= 'left')
print(train_df.shape)
train_df.head()

(20216100, 16)


,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,106,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,102,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,89,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,100,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,73,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0


In [39]:
train_df.isna().sum()

building_id               0
meter                     0
timestamp                 0
meter_reading             0
site_id                   0
primary_use               0
square_feet               0
year_built                0
floor_count               0
air_temperature       90495
cloud_coverage        90495
dew_temperature       90495
precip_depth_1_hr     90495
sea_level_pressure    90495
wind_direction        90495
wind_speed            90495
dtype: int64

In [40]:
train_df[train_df['air_temperature'].isna()].head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
722,565,0,2016-01-01 00:00:00,7.250,4,Education,15326,52,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
723,566,0,2016-01-01 00:00:00,0.519,4,Education,2010,55,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
724,569,0,2016-01-01 00:00:00,246.000,4,Education,86091,62,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
725,570,0,2016-01-01 00:00:00,79.488,4,Education,193202,62,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
726,571,0,2016-01-01 00:00:00,17.000,4,Education,47954,78,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
ww = weather_train_df[weather_train_df['site_id'] == 4]
ww.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
35110,4,2016-01-01 01:00:00,9.4,0.0,-2.2,0.0,1021.4,360.0,3.1
35111,4,2016-01-01 02:00:00,8.3,0.0,-2.8,0.0,1021.6,0.0,0.0
35112,4,2016-01-01 03:00:00,7.8,0.0,-2.8,0.0,1021.7,90.0,1.5
35113,4,2016-01-01 04:00:00,5.0,0.0,-0.6,0.0,1021.8,0.0,0.0
35114,4,2016-01-01 05:00:00,5.6,0.0,-1.1,0.0,1021.6,130.0,3.1


In [42]:
ww[ww['timestamp'] == '2016-01-01 00:00:00']

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed


In [43]:
train_df.fillna(-100,inplace=True)

In [44]:
train_df.isna().sum()

building_id           0
meter                 0
timestamp             0
meter_reading         0
site_id               0
primary_use           0
square_feet           0
year_built            0
floor_count           0
air_temperature       0
cloud_coverage        0
dew_temperature       0
precip_depth_1_hr     0
sea_level_pressure    0
wind_direction        0
wind_speed            0
dtype: int64

In [45]:
test_df = pd.merge(test_df,weather_test_df,on=['site_id','timestamp'],how= 'left')
print(test_df.shape)
test_df.head()

(41697600, 16)


,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,106,-1,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,102,-1,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,89,-1,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,100,-1,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,73,-1,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6


In [46]:
test_df.isna().sum()

row_id                     0
building_id                0
meter                      0
timestamp                  0
site_id                    0
primary_use                0
square_feet                0
year_built                 0
floor_count                0
air_temperature       199029
cloud_coverage        199029
dew_temperature       199029
precip_depth_1_hr     199029
sea_level_pressure    199029
wind_direction        199029
wind_speed            199029
dtype: int64

In [47]:
test_df.fillna(-100,inplace = True)
test_df.shape

(41697600, 16)

In [48]:
del weather_test_df,weather_train_df,building_meatdata_Df

In [49]:
train_df.dtypes

building_id             int64
meter                   int64
timestamp              object
meter_reading         float64
site_id                 int64
primary_use            object
square_feet             int64
year_built               int8
floor_count              int8
air_temperature       float64
cloud_coverage        float64
dew_temperature       float64
precip_depth_1_hr     float64
sea_level_pressure    float64
wind_direction        float64
wind_speed            float64
dtype: object

In [51]:
train_df.to_csv('merged_train.csv',index = False)
test_df.to_csv('merged_test.csv',index = False)

**converting primary use to category and timestamp to date format**

In [2]:
def change_dtype(df):
    int_cols = ['building_id','meter','site_id','primary_use','square_feet','year_built','floor_count',
                'year','month','day','hour','minute','second']
    for col in df.columns:
        if col != 'timestamp':
            mx = df[col].max()
            mn = df[col].min()
            isInt = True if col in int_cols else False

            if isInt:
                if mn >= 0:
                    if mx <= 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx <= 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx <= 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
            else:
                df[col] = df[col].astype(np.float32)
        else:
            pass
    
    return df

**------------------------------------ Train data ---------------------------------------**

In [3]:
train_df = pd.read_csv('merged_train.csv')
print(train_df.shape)
train_df.head()

(20216100, 16)


,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,106,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,102,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,89,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,100,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,73,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0


In [4]:
train_df['primary_use']=  train_df['primary_use'].astype('category')
train_df['primary_use'] = train_df['primary_use'].cat.codes


In [5]:
train_df[['date','time']] = train_df['timestamp'].str.split(expand = True)
train_df.drop('timestamp',axis = 1,inplace = True)
train_df.head(2)

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,date,time
0,0,0,0.0,0,0,7432,106,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,2016-01-01,00:00:00
1,1,0,0.0,0,0,2720,102,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,2016-01-01,00:00:00


In [6]:
train_df['date'] = pd.to_datetime(train_df['date'],format='%Y-%m-%d')
train_df['time'] = pd.to_datetime(train_df['time'],format='%H:%M:%S')
train_df.head(2)

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,date,time
0,0,0,0.0,0,0,7432,106,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,2016-01-01,1900-01-01
1,1,0,0.0,0,0,2720,102,-1,25.0,6.0,20.0,-1.0,1019.7,0.0,0.0,2016-01-01,1900-01-01


In [7]:
train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['day'] = train_df['date'].dt.day

train_df['hour'] = train_df['time'].dt.hour
train_df['minute'] = train_df['time'].dt.minute
train_df['second'] = train_df['time'].dt.second
train_df.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,wind_direction,wind_speed,date,time,year,month,day,hour,minute,second
0,0,0,0.0,0,0,7432,106,-1,25.0,6.0,...,0.0,0.0,2016-01-01,1900-01-01,2016,1,1,0,0,0
1,1,0,0.0,0,0,2720,102,-1,25.0,6.0,...,0.0,0.0,2016-01-01,1900-01-01,2016,1,1,0,0,0
2,2,0,0.0,0,0,5376,89,-1,25.0,6.0,...,0.0,0.0,2016-01-01,1900-01-01,2016,1,1,0,0,0
3,3,0,0.0,0,0,23685,100,-1,25.0,6.0,...,0.0,0.0,2016-01-01,1900-01-01,2016,1,1,0,0,0
4,4,0,0.0,0,0,116607,73,-1,25.0,6.0,...,0.0,0.0,2016-01-01,1900-01-01,2016,1,1,0,0,0


In [8]:
train_df.drop(['date','time'],axis = 1,inplace = True)

In [9]:
train_df.shape

(20216100, 21)

In [10]:
train_df = change_dtype(train_df)
train_df.dtypes

building_id            uint16
meter                   uint8
meter_reading         float32
site_id                 uint8
primary_use             uint8
square_feet            uint32
year_built               int8
floor_count              int8
air_temperature       float32
cloud_coverage        float32
dew_temperature       float32
precip_depth_1_hr     float32
sea_level_pressure    float32
wind_direction        float32
wind_speed            float32
year                   uint16
month                   uint8
day                     uint8
hour                    uint8
minute                  uint8
second                  uint8
dtype: object

In [11]:
train_df.year.value_counts()

2016    20216100
Name: year, dtype: int64

In [12]:
train_df.month.value_counts()

10    1747349
5     1743444
7     1739087
12    1726819
8     1721185
1     1708850
9     1694451
6     1689252
11    1689024
4     1686336
3     1575403
2     1494900
Name: month, dtype: int64

In [13]:
Y = train_df['meter_reading']
X = train_df.drop(['meter_reading'],axis = 1)

In [15]:
Y.to_csv('y.csv',index = False)
X.to_csv('x.csv',index = False)

/home/paperspace/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [16]:
del train_df

**------------------------- Load data ----------------------------**

In [17]:
X = pd.read_csv('x.csv')
Y = pd.read_csv('y.csv',header=None)

In [18]:
X = change_dtype(X)

In [19]:
X.shape,Y.shape

((20216100, 20), (20216100, 1))

In [20]:
X = np.array(X)
Y = np.squeeze(np.array(Y))

In [21]:
from sklearn.model_selection import KFold,TimeSeriesSplit,train_test_split

In [22]:
#loss function

def RMSLE(y_t,y_p):
    if len(y_t) == len(y_p):
        pred_log = np.log(y_p+1)
        act_log = np.log(y_t+1)
        rms = np.square(pred_log - act_log)
        rms = np.sum(rms)/len(rms)
        return np.sqrt(rms)
    else:
       raise ValueError('Length of y_true and y_pred should be same') 

### LGBM model

In [23]:
import lightgbm as lgb

In [24]:
params = {'num_leaves': 31,
          'min_child_weight': 0.001,
          'objective': 'regression',
          'max_depth': -1,
          'learning_rate': 0.1,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'rmse',
          "verbosity": -1,
          'reg_alpha': 0.0,
          'reg_lambda': 0.0,
          'random_state': 47,
          "n_jobs" : -1
         }

In [27]:
folds = KFold(n_splits=5,random_state=2019)
prediction = 0
rmsle = []

for fold,(train_idx,valid_idx) in enumerate(folds.split(X,Y)):
    print('Training on fold {}'.format(fold + 1))
    X_tr,Y_tr = X[train_idx],Y[train_idx]
    X_va,Y_va = X[valid_idx],Y[valid_idx]
    
    clf = lgb.LGBMRegressor(**params,n_estimators=10000)
    clf.fit(X_tr,Y_tr,eval_set = [(X_tr,Y_tr),(X_va,Y_va)],
           eval_metric = 'rmse' ,verbose = 500,early_stopping_rounds = 500)
    
    prediction += clf.predict(X_va) / folds.n_splits
    error = RMSLE(Y_va,prediction)
    
    print('RMSLE for fold '+str(fold+1) + ' : ' + str(error))
    rmsle.append(error)
print('-'*20)
print('Trianing Completed')
print('Mean RMSLE: ',np.mean(rmsle))
print('-'*20)

Training on fold 1
Training until validation scores don't improve for 500 rounds
[500]	training's rmse: 15715.3	valid_1's rmse: 152316
Early stopping, best iteration is:
[5]	training's rmse: 98916.7	valid_1's rmse: 107538
RMSLE for fold 1 : 2.823984989638263
Training on fold 2
Training until validation scores don't improve for 500 rounds
[500]	training's rmse: 13754.5	valid_1's rmse: 128657
Early stopping, best iteration is:
[17]	training's rmse: 34593.4	valid_1's rmse: 120537
RMSLE for fold 2 : 2.9604661335436107
Training on fold 3
Training until validation scores don't improve for 500 rounds
[500]	training's rmse: 18221.9	valid_1's rmse: 145402
Early stopping, best iteration is:
[8]	training's rmse: 75946.4	valid_1's rmse: 117470
RMSLE for fold 3 : 3.0229609704923774
Training on fold 4
Training until validation scores don't improve for 500 rounds
[500]	training's rmse: 19749	valid_1's rmse: 30525.2
Early stopping, best iteration is:
[1]	training's rmse: 155728	valid_1's rmse: 8411.22

In [28]:
clf.booster_.save_model('lgbm_!.txt')

**---------------------------------- Test Data -------------------------------------------**

In [30]:
del X,Y,X_tr,Y_tr,X_va,Y_va

In [31]:
test_df = pd.read_csv('merged_test.csv')
print(test_df.shape)
test_df.head()

(41697600, 16)


,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,106,-1,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,102,-1,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,89,-1,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,100,-1,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,73,-1,17.8,4.0,11.7,-1.0,1021.4,100.0,3.6


In [32]:
test_df['primary_use']=  test_df['primary_use'].astype('category')
test_df['primary_use'] = test_df['primary_use'].cat.codes

In [33]:
test_df = change_dtype(test_df)

In [34]:
test_df[['date','time']] = test_df['timestamp'].str.split(expand = True)
test_df.drop('timestamp',axis = 1,inplace = True)
test_df.head(2)

,row_id,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,date,time
0,0.0,0,0,0,0,7432,106,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017-01-01,00:00:00
1,1.0,1,0,0,0,2720,102,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017-01-01,00:00:00


In [35]:
test_df['date'] = pd.to_datetime(test_df['date'],format='%Y-%m-%d')
test_df['time'] = pd.to_datetime(test_df['time'],format='%H:%M:%S')
test_df.head(2)

,row_id,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,date,time
0,0.0,0,0,0,0,7432,106,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017-01-01,1900-01-01
1,1.0,1,0,0,0,2720,102,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017-01-01,1900-01-01


In [36]:
test_df['year'] = test_df['date'].dt.year
test_df['month'] = test_df['date'].dt.month
test_df['day'] = test_df['date'].dt.day

test_df['hour'] = test_df['time'].dt.hour
test_df['minute'] = test_df['time'].dt.minute
test_df['second'] = test_df['time'].dt.second
test_df.head()

,row_id,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,...,wind_direction,wind_speed,date,time,year,month,day,hour,minute,second
0,0.0,0,0,0,0,7432,106,-1,17.799999,4.0,...,100.0,3.6,2017-01-01,1900-01-01,2017,1,1,0,0,0
1,1.0,1,0,0,0,2720,102,-1,17.799999,4.0,...,100.0,3.6,2017-01-01,1900-01-01,2017,1,1,0,0,0
2,2.0,2,0,0,0,5376,89,-1,17.799999,4.0,...,100.0,3.6,2017-01-01,1900-01-01,2017,1,1,0,0,0
3,3.0,3,0,0,0,23685,100,-1,17.799999,4.0,...,100.0,3.6,2017-01-01,1900-01-01,2017,1,1,0,0,0
4,4.0,4,0,0,0,116607,73,-1,17.799999,4.0,...,100.0,3.6,2017-01-01,1900-01-01,2017,1,1,0,0,0


In [37]:
del test_df['date']

In [38]:
del test_df['time']

In [39]:
test_df = change_dtype(test_df)
test_df.dtypes

row_id                float32
building_id            uint16
meter                   uint8
site_id                 uint8
primary_use             uint8
square_feet            uint32
year_built               int8
floor_count              int8
air_temperature       float32
cloud_coverage        float32
dew_temperature       float32
precip_depth_1_hr     float32
sea_level_pressure    float32
wind_direction        float32
wind_speed            float32
year                   uint16
month                   uint8
day                     uint8
hour                    uint8
minute                  uint8
second                  uint8
dtype: object

In [40]:
test_df.drop(['row_id'],axis = 1,inplace=True)

In [41]:
test_df.shape

(41697600, 20)

In [21]:
test_df.year.value_counts()

2018    20848800
2017    20848800
Name: year, dtype: int64

In [22]:
test_df[test_df['year'] == 2018].month.value_counts()

12    1770720
10    1770720
8     1770720
7     1770720
5     1770720
3     1770720
1     1770720
11    1713600
9     1713600
6     1713600
4     1713600
2     1599360
Name: month, dtype: int64

In [23]:
test_df[test_df['year'] == 2017].month.value_counts()

12    1770720
10    1770720
8     1770720
7     1770720
5     1770720
3     1770720
1     1770720
11    1713600
9     1713600
6     1713600
4     1713600
2     1599360
Name: month, dtype: int64

In [45]:
del prediction,rmsle

In [55]:
test_df.head()

,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,year,month,day,hour,minute,second
0,0,0,0,0,7432,106,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017,1,1,0,0,0
1,1,0,0,0,2720,102,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017,1,1,0,0,0
2,2,0,0,0,5376,89,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017,1,1,0,0,0
3,3,0,0,0,23685,100,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017,1,1,0,0,0
4,4,0,0,0,116607,73,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017,1,1,0,0,0


In [56]:
test_df[:3]

,building_id,meter,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,year,month,day,hour,minute,second
0,0,0,0,0,7432,106,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017,1,1,0,0,0
1,1,0,0,0,2720,102,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017,1,1,0,0,0
2,2,0,0,0,5376,89,-1,17.799999,4.0,11.7,-1.0,1021.400024,100.0,3.6,2017,1,1,0,0,0


In [57]:
for i in range(10):
    l = len(test_df)
    np.save(str(i+1)+'.npy',clf.predict(np.array(test_df[i:(i+1)*l//10])))
    test_df.drop(test_df.index[i:(i+1)*l//10])

MemoryError: 

### Sample Submission

In [5]:
sub = pd.read_csv('sample_submission.csv')
print(sub.shape)
sub.head()

(41697600, 2)


,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
